Import

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

from change_point_model import (
    prepare_data, run_change_point_model, 
    summarize_results, get_change_point, plot_change_point,
    calculate_impact, match_events)
from data_analysis import load_data
import pandas as pd

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl
WARN: Could not locate executable gfortran
WARN: Could not locate executable f95
WARN: Could not locate executable g95
WARN: Could not locate executable efort
WARN: Could not locate executable efc
WARN: Could not locate executable flang
WARN: don't know how to compile Fortran code on platform 'nt'


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


ImportError: cannot import name 'Tester' from 'numpy.testing' (e:\kaim projec\Change point analysis and statistical modelling\.venv\Lib\site-packages\numpy\testing\__init__.py)

load data

In [ ]:
df = load_data("../data/BrentOilPrices.csv")

Prepare log returns

In [ ]:
data, date_index = prepare_data(df)

Run Bayesian Change Point Model

In [ ]:
trace = run_change_point_model(data)

Summarize results

In [ ]:
summarize_results(trace)

Get change point

In [ ]:
tau_index, change_point_date = get_change_point(trace, date_index)
print(f"Most probable change point index: {tau_index}")
print(f"Most probable change point date: {change_point_date}")

Plot change point

In [ ]:
plot_change_point(data, date_index, change_point_date)


Calculate impact

In [ ]:
mu_1_mean, mu_2_mean, impact_pct = calculate_impact(trace)
print(f"Mean before: {mu_1_mean:.6f}, Mean after: {mu_2_mean:.6f}")
print(f"Impact: {impact_pct:.2f}% change")

Load event dataset

In [ ]:
events_df = pd.read_csv("../data/key_events.csv")

Match with event

In [ ]:
event_name, event_date = match_event(change_point_date, events_df)
if event_name:
    print(f"Closest event: {event_name} on {event_date.date()}")
else:
    print("No matching event found within the threshold")

Summary table

In [ ]:
summary = {
    "Change Point Date": [change_point_date],
    "Event": [event_name if event_name else "No Event"],
    "Event Date": [event_date if event_name else None],
    "Mean Before": [mu_1_mean],
    "Mean After": [mu_2_mean],
    "Impact (%)": [impact_pct]
}

summary_df = pd.DataFrame(summary)
summary_df.to_csv("../results/change_point_summary.csv", index=False)
summary_df
